# Object-oriented FaIR

Why do classes make absolutely zero sense?

In [ ]:
from abc import ABC

import numpy as np
import pandas as pd

from fair21.defaults.gases import iirf_horizon, pre_industrial_concentration
from fair21.defaults import f_gas_list, montreal_gas_list, slcf_list, gas_list, n_gas_boxes
from fair21.exceptions import MissingInputError, IncompatibleConfigError, PartitionFractionError, LifetimeError

`Specie` is an abstract base class. Everything that we care about for climate is a `Specie`, but we should encourage users to use derived classes like `GreenhouseGas`

In [ ]:
class Specie(ABC):
    
    def __init__(self, name, tropospheric_adjustment=1):
        self.name = name
        self.tropospheric_adjustment=tropospheric_adjustment
    
    def set_index(self, index):
        self.index = index

In [ ]:
class FAIR():
    def run():
        pass

In [ ]:
class GreenhouseGas(Specie):
    
    def __init__(self, name, molecular_weight, lifetime, radiative_efficiency, **kwargs):
        tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0)
        super().__init__(name, tropospheric_adjustment)
        
        # move the below to input verification method
        self.molecular_weight = molecular_weight
        self.lifetime = lifetime
        self.radiative_efficiency = radiative_efficiency
        if np.ndim(self.lifetime) == 1:
            lifetime = np.asarray(lifetime)
            # should we enforce whether strictly decreasing or not?
            partition_fraction = kwargs.get('partition_fraction')
            if partition_fraction is None:
                raise MissingInputError('specify `partition_fraction` if specifying more than one `lifetime`') # custom exception needed
            if len(partition_fraction) != len(lifetime):
                raise IncompatibleConfigError('`partition_fraction` and `lifetime` are different shapes') # custom exception needed
            partition_fraction = np.asarray(partition_fraction)
            if ~np.isclose(np.sum(partition_fraction), 1):
                raise PartitionFractionError('partition_fraction should sum to 1') # custom exception needed
        elif np.ndim(self.lifetime) > 1:
            raise LifetimeError('`lifetime` array dimension is greater than 1')
        else:
            partition_fraction=1
        self.partition_fraction=partition_fraction
        self.emissions = kwargs.get('emissions')
        self.concentrations = kwargs.get('concentrations')
        self.forcing = kwargs.get('forcing')
        self.g0, self.g1 = self.calculate_g(partition_fraction, lifetime, iirf_horizon=iirf_horizon)
        self.iirf_0 = kwargs.get('iirf_0', self.lifetime_to_iirf_0(lifetime, partition_fraction, iirf_horizon=iirf_horizon))
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0)
        self.iirf_temperature = kwargs.get('iirf_temperature', 0)
        self.output_forcing=True
        self.pre_industrial_concentrations=0

    
    @staticmethod
    def calculate_g(partition_fraction, lifetime, iirf_horizon=100):
        g1 = np.sum(partition_fraction * lifetime * (1 - (1 + iirf_horizon/lifetime) * np.exp(-iirf_horizon/lifetime)))
        g0 = np.exp(-1 * np.sum(partition_fraction*lifetime*(1 - np.exp(-iirf_horizon/lifetime)))/g1)
        return g0, g1
    
    @staticmethod
    def lifetime_to_iirf_0(lifetime, partition_fraction=1,iirf_horizon=100):
        return np.sum(lifetime * (1 - np.exp(-iirf_horizon / lifetime)) * partition_fraction)

In [ ]:
class CO2(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.009)
        self.lifetime = kwargs.pop('lifetime', np.array([1e9, 394.4, 36.54, 4.304]))
        self.partition_fraction = kwargs.pop('partition_fraction', np.array([0.2173, 0.2240, 0.2824, 0.2763]))
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.05)
        self.radiative_efficiency = kwargs.pop('radiative_efficiency', 0.012895)  # Hodnebrog 2020: we should calculate based on Meinshausen formula and provide this as a convenience method
        
        super().__init__(
            name="CO2",
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            partition_fraction=self.partition_fraction,
            tropospheric_adjustment=self.tropospheric_adjustment,
            radiative_efficiency=self.radiative_efficiency,
            **kwargs
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Gt CO2/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppm')
        self.iirf_0 = kwargs.get('iirf_0', 29)
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0.00846)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.000819)
        self.iirf_temperature = kwargs.get('iirf_temperature', 4)
        self.pre_industrial_concentrations=kwargs.get('pre_industrial_concentrations', 278.3)

In [ ]:
co2 = CO2(
#    emissions=np.ones(50) * 10,
    molecular_weight=43,
    iirf_0=32
)

In [ ]:
vars(co2)

In [ ]:
vars(co2)

In [ ]:
class Halogen(GreenhouseGas):
    def __init__(self, cl_atoms, br_atoms, fractional_release, **kwargs):
        super().__init__(**kwargs)
        self.cl_atoms = cl_atoms
        self.br_atoms = br_atoms
        self.fractional_release = fractional_release
        #self.radiative_efficiency = kwargs.get('radiative_efficiency')
        self.ozone_radiative_efficiency = -1.25e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentrations_unit = 'ppt'
        self.pre_industrial_concentrations=kwargs.get('pre_industrial_concentrations', 0)
        
#     def calculate_eesc():
#         pass
    
#         eesc = calculate_eesc(
#         concentration,
#         pre_industrial_concentration,
#         fractional_release,
#         cl_atoms,
#         br_atoms,
#         species_index_mapping,
#         br_cl_ratio=br_cl_ratio,
#     )

In [ ]:
# archetype for default
cfc_11 = Halogen(name='CFC-11', tropospheric_adjustment=0.13, lifetime=45, fractional_release=0.23, cl_atoms=2, br_atoms=0, molecular_weight=500, radiative_efficiency=0.2)

In [ ]:
vars(cfc_11)

In [ ]:
class FGas(GreenhouseGas):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentrations_unit = 'ppt'
        self.pre_industrial_concentrations=0

In [ ]:
hfc134a = FGas(name='HFC-134a', lifetime=23, radiative_efficiency=0.3, molecular_weight=140)

In [ ]:
vars(hfc134a)

In [ ]:
class CH4(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 16.043)
        self.lifetime = kwargs.pop('lifetime', 8.25)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', -0.14)
        
        super().__init__(
            name='CH4',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            #partition_fraction=self.partition_fraction,
            **kwargs
        )
        self.ozone_radiative_efficiency = 1.75e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt CH4/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.00032)
        self.iirf_temperature = kwargs.get('iirf_temperature', -0.3)
        self.pre_industrial_concentrations=kwargs.get('pre_industrial_concentrations', 729.2)

In [ ]:
ch4 = CH4(iirf_0 = 32)

In [ ]:
vars(ch4)

In [ ]:
class N2O(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.013)
        self.lifetime = kwargs.pop('lifetime', 109)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.07)
        
        super().__init__(
            name='N2O',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            **kwargs
        )
        self.ozone_radiative_efficiency = 7.10e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt N2O/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')
        self.pre_industrial_concentrations = kwargs.get('pre_industrial_concentrations', 270.1)

In [ ]:
n2o = N2O()
vars(n2o)

In [ ]:
# now let's say we want to simulate UKESM and say CH4 is indeed an aerosol precursor. Can we do this?
# it doesn't complain...
ch4 = CH4(ari_radiative_efficiency=0.4)

In [ ]:
#...but the argument is unused and doesnt appear as instance of CH4. This needs to be sorted out
vars(ch4)

In [ ]:
class ShortLivedForcer(Specie):
    
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        self.emissions = kwargs.get('emissions')
        self.forcing = kwargs.get('forcing')
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt {}/yr'.format(self.name.replace("-", "")))
        self.pre_industrial_emissions = kwargs.get('pre_industrial_emissions', 0)

In [ ]:
# we'll make ERFari a direct property of the species, and ERFaci is a class

class AerosolPrecursor(ShortLivedForcer):
    def __init__(self, ari_radiative_efficiency, **kwargs):
        super().__init__(**kwargs)
        self.output_forcing = True

In [ ]:
class Sulfur(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.0036167830509091486)
        super().__init__(
            name='Sulfur',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt SO2/yr')

In [ ]:
sulfur = Sulfur()
vars(sulfur)

In [ ]:
class BC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', 0.0507748226795483)
        super().__init__(
            name='BC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
class OC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.006214374446217472)
        super().__init__(
            name='OC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
bc = BC()
vars(bc)

In [ ]:
oc = OC()
vars(oc)

In [ ]:
class Ozone(Specie):
    def __init__(self, **kwargs):
        super().__init__(name='Ozone')
        self.forcing_output=True
        self.temperature_feedback = -0.037

In [ ]:
ozone = Ozone()
vars(ozone)

In [ ]:
class AerosolCloudInteractions():
    def __init__(self, **kwargs):
        super().__init__(name='Aerosol-cloud interactions')
        self.forcing_output=True
        #scale =
        #shape_sulfur = 
        #shape_bcoc = 

In [ ]:
# class(Scenario):
    
#     def __init__(self, species_list):
        

In [ ]:
species_list = [co2, ch4, n2o]

In [ ]:
run_mode = {
    'forcing driven' : False,
    'CO2' : 'emissions',
    'CH4' : 'emissions',
    'N2O' : 'emissions',
    'F-Gases': None,#'emissions',
    'Montreal Gases': None,#'emissions',
    'Aerosol-radiation interactions' : None,#'emissions',
    'Aerosol-cloud interactions' : None,#'emissions',
    'Ozone' : None,#'emissions',
    'Land use' : None, #'emissions',
    'Stratospheric water vapour':  None,#'emissions',
    'Contrails' : None,#'emissions',
    'Black carbon on snow' : None,#'emissions',
    'Solar': None,#'forcing',
    'Volcanic': None,#'forcing'
}

valid_run_modes = {
    'forcing driven' : (False, True),
    'CO2' : ('emissions', 'concentrations', 'forcing', None),
    'CH4' : ('emissions', 'concentrations', 'forcing', None),
    'N2O' : ('emissions', 'concentrations', 'forcing', None),
    'F-Gases' : ('emissions', 'concentrations', 'forcing', None),
    'Montreal Gases' : ('emissions', 'concentrations', 'forcing', None),
    'Aerosol-radiation interactions' : ('emissions', 'forcing', None),
    'Aerosol-cloud interactions': ('emissions', 'forcing', None),
    'Ozone' : ('emissions', 'forcing', None),
    'Land use' : ('emissions', 'forcing', None),
    'Stratospheric water vapour' : ('emissions', 'forcing', None),
    'Contrails' : ('emissions', 'forcing', None),
    'Black carbon on snow' : ('emissions', 'forcing', None),
    'Solar' : ('forcing', None),
    'Volcanic' : ('forcing', None),
}

In [ ]:
required_species_emissions_or_concentrations = {
    'forcing driven' : None,
    'CO2' : ['CO2', 'N2O'],
    'CH4' : ['CH4', 'N2O'],
    'N2O' : ['CO2', 'CH4', 'N2O'],
    'F-Gases' : None,
    'Montreal Gases' : None,
    'Aerosol-radiation interactions': None,
    'Aerosol-cloud interactions' : ['Sulfur', 'BC', 'OC'],
    'Ozone' : ['CFC-11'],
    'Land use' : ['CO2|AFOLU'],
    'Stratospheric water vapour' : ['CH4'],
    'Contrails' : ['NOx|Aviation'],
    'Black carbon on snow' : ['BC'],
    'Solar' : ['Solar'],
    'Volcanic' : ['Volcanic'],
}

required_species = []

for mode in run_mode:
    if required_species_emissions_or_concentrations[mode] is None:
        continue
    if run_mode[mode] in ('emissions', 'concentrations') :
        required_species.extend(required_species_emissions_or_concentrations[mode])
    if mode in ('Solar', 'Volcanic'):
        if run_mode[mode]=='forcing':
            required_species.extend(required_species_emissions_or_concentrations[mode])

required_species = list(set(required_species))
required_species

In [ ]:
# check for duplicates
species_names = []
for specie in species_list:
    if specie.name in species_names:
        raise DuplicatedSpeciesError(f'{specie.name} is non-unique in the scenario')
    species_names.append(specie.name)

# check everything required is defined in the scenario
for specie_name in required_species:
    if specie_name not in species_names:
        raise MissingInputError(f'{specie_name} is required but not in scenario')

In [ ]:
species_names

In [ ]:
ozone_emissions_indices = []
ozone_concentrations_indices = []
ari_indices = []
ghg_indices = []
index_mapping = {}

for ispec, specie in enumerate(species_list):
    index_mapping[specie.name] = ispec
    specie.set_index(ispec)
    if hasattr(specie, "ozone_radiative_efficiency") and isinstance(specie, ShortLivedForcer):
        ozone_emissions_indices.append(ispec)
    if hasattr(specie, "ozone_radiative_efficiency") and isinstance(specie, GreenhouseGas):
        ozone_concentrations_indices.append(ispec)
    if isinstance(specie, GreenhouseGas):
        ghg_indices.append(ispec)
    if isinstance(specie, AerosolPrecursor):
        ari_indices.append(ispec)

In [ ]:
ozone_emissions_indices

In [ ]:
ozone_concentrations_indices

In [ ]:
ghg_indices

In [ ]:
ari_indices

In [ ]:
if run_mode['Aerosol-cloud interactions']=='emissions':
    aci_indices = [index_mapping[specie] for specie in required_species_emissions_or_concentrations['Aerosol-cloud interactions']] # in that order
    # then we can choose different treatments in  required_species_emissions_or_concentrations['Aerosol-cloud interactions'], e.g. stevens
else:
    aci_indices = []

In [ ]:
aci_indices

In [ ]:
index_mapping

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
n_timesteps = 751
n_scenarios = len(scenarios)

n_species = len(species_list)
n_species

In [ ]:
class Scenario():
    
    # species_list is a list of objects
    # species_names is a list of strings
        
    def __init__(self, species=None):
        self.species = {}
        
        if hasattr(species, '__iter__'):
            # this will automatically update/avoid duplicates
            self.species = {specie.name: specie for specie in species}
        elif species is not None:
            self.species = {species.name: species}

#         # if running emis or conc driven and >0 and <3 of CO2, CH4 and N2O are defined, fill in the others
#         # with pre-industrial levels
#         if run_mode["CO2"] in ("emissions", "concentrations") or run_mode["CH4"] in ("emissions", "concentrations") or run_mode["N2O"] in ("emissions", "concentrations"):
#             for gas in ("CO2", "CH4", "N2O"):
#                 if "CO2" not in self.species:
#                     self.species["CO2"] = CO2(emissions = 0, concentrations = pre_industrial_concentration["CO2"])
#                 if "CH4" not in self.species:
#                     self.species["CH4"] = CH4(emissions = 0, concentrations = pre_industrial_concentration["CH4"])
#                 if "N2O" not in self.species:
#                     self.species["N2O"] = N2O(emissions = 0, concentrations = pre_industrial_concentration["N2O"])

    def _assign_indices(self):
        """Assign a unique index to each specie included in the scenario."""
        self.species_index_mapping = {}
        for ispec, specie in enumerate(self.species):
            self.species_index_mapping[specie] = ispec
            

    def _initialise_arrays(self, n_scenarios, n_species, n_timesteps):
        self.emissions_array = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan
        self.concentration_array = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan
        self.forcing_array = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan
        self.g0 = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.g1 = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.alpha_lifetime = np.ones((n_scenarios, n_species, 1, 1))
        self.airborne_emissions = np.zeros((n_scenarios, n_species, 1, 1))
        self.gas_boxes = np.zeros((n_scenarios, n_species, 1, n_gas_boxes))
        self.iirf_0_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.iirf_cumulative_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.iirf_temperature_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.iirf_airborne_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.burden_per_emission_array = np.ones((1, n_species, 1, 1)) * np.nan
        self.lifetime_array = np.ones((n_scenarios, n_species, 1, n_gas_boxes)) * np.nan
        self.pre_industrial_emissions_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.pre_industrial_concentration_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        self.partition_fraction_array = np.zeros((n_scenarios, n_species, 1, n_gas_boxes))
        self.natural_emissions_adjustment_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
        
        
        # This is an ugly amount of repetition, but I don't think it's possible to setattr on a slice of
        # a numpy array without using eval, which is even uglier
        for ispec, specie in enumerate(self.species):
            if hasattr(self.species[specie], 'emissions'):
                self.emissions_array[0, ispec, :, 0] = self.species[specie].emissions
            if hasattr(self.species[specie], 'concentration'):
                self.concentration_array[0, ispec, :, 0] = self.species[specie].concentration
            if hasattr(self.species[specie], 'forcing'):
                self.forcing_array[0, ispec, :, 0] = self.species[specie].forcing
            if isinstance(self.species[specie], GreenhouseGas):
                self.lifetime_array[:, ispec, :, :] = self.species[specie].lifetime
                self.partition_fraction_array[:, ispec, :, :] = self.species[specie].partition_fraction
                self.iirf_0_array[:, ispec, :, :] = self.species[specie].iirf_0
                self.iirf_cumulative_array[:, ispec, :, :] = self.species[specie].iirf_cumulative
                self.iirf_temperature_array[:, ispec, :, :] = self.species[specie].iirf_temperature
                self.iirf_airborne_array[:, ispec, :, :] = self.species[specie].iirf_airborne
                self.burden_per_emission_array[:, ispec, :, :] = self.species[specie].burden_per_emission
                self.pre_industrial_concentration_array[:, ispec, :, :] = self.species[specie].pre_industrial_concentration
                self.natural_emissions_adjustment_array[:, ispec, :, :] = self.species[specie].natural_emissions_adjustment
                self.radiative_efficiency[:, ispec, :, :] = self.species[specie].radiative_efficiency
                self.g0[:, ispec, :, :] = self.species[specie].g0
                self.g1[:, ispec, :, :] = self.species[specie].g1
            if isinstance(self.species[specie], Halogen):
                self.fractional_release_array[:, ispec, :, :] = self.species[specie].fractional_release
                self.br_atoms_array[:, ispec, :, :] = self.species[specie].br_atoms
                self.cl_atoms_array[:, ispec, :, :] = self.species[specie].cl_atoms
                
    # this is the forward run - and should be generalised
    def run(self):
        
        n_species = len(self.species)
        n_scenarios = 1  # just for now
        n_timesteps = 751 # also a placeholder
        
        # initialise arrays. Using arrays makes things run quicker
        self._assign_indices()
        self._initialise_arrays(n_scenarios, n_species, n_timesteps)

        


# cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)

# # this one for the general forcing
# tropospheric_adjustment_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
# radiative_efficiency_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
# erf = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan

# # ozone
# fractional_release_array = np.zeros((1, n_species, 1, 1)) # gonna assume this doesn't change
# cl_atoms_array = np.zeros((1, n_species, 1, 1))
# br_atoms_array = np.zeros((1, n_species, 1, 1))
# o3_radiative_efficiency_array = np.zeros((1, n_species, 1, 1))

# for ispec, specie in enumerate(species_list):
#     # options pertaining to SLCFs
#     if specie in slcf_list:
#         pre_industrial_emissions_array[:, ispec, :, :] = pre_industrial_emissions[specie]
    
#     # options pertaining to GHGs
#     if specie in gas_list:

        
#     # options pertaining only to minor GHGs
#     if specie in minor_gas_list:
#         radiative_efficiency_array[:, ispec, :, :] = radiative_efficiency[specie]
        

#     # options pertaining to all species
#     tropospheric_adjustment_array[:, ispec, :, :] = tropospheric_adjustment[specie]
#     o3_radiative_efficiency_array[:, ispec, :, :] = o3_radiative_efficiency[specie]

# g0, g1 = calculate_g(lifetime_array, partition_fraction_array)
        
    # times
    # configs

In [ ]:
scen = Scenario()

In [ ]:
scen

In [ ]:
vars(scen)

In [ ]:
scen = Scenario(species=(co2, ch4, n2o))

In [ ]:
scenario = 'ssp245'

In [ ]:
# grab some emissions
emissions = {}

df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    emissions[scenario] = {}
    for igas, gas in enumerate(gas_list):
        gas_rcmip_name = gas.replace("-", "")
        emissions[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.T.squeeze()
    
        # CO2 and N2O units need to behave
        if gas in ('CO2', 'N2O'):
            emissions[scenario][gas] = emissions[scenario][gas] / 1000

In [ ]:
for gas in ['CO2', 'CH4', 'N2O']:
    scen.species[gas].emissions = emissions['ssp245'][gas]

In [ ]:
vars(scen.species['CH4'])

In [ ]:
scen.run()

In [ ]:
vars(scen)

In [ ]:
# scenario = 'ssp245'
# from fair21.defaults import f_gas_list, montreal_gas_list, slcf_list
# for specie in f_gas_list + montreal_gas_list + slcf_list:
#     print (specie)